In [26]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.metrics import accuracy_score
from scipy.stats import mode
import pandas as pd
import numpy as np
import errno
import copy
from copy import deepcopy
import csv
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [27]:
########## WITHOUT REWEIGHING #############
stat_par = []
disp_imp = []
eq_opp_diff = []
avg_odds_diff = []
theil = []
acc = []

########## WITH REWEIGHING #############
stat_par_reweigh = []
disp_imp_reweigh = []
eq_opp_diff_reweigh = []
avg_odds_diff_reweigh = []
theil_reweigh = []
acc_reweigh = []

# number of times you want to run this
num_runs = 4

In [28]:
def preprocess_germandataset(df):
    def group_credit_hist(x):
        if x in ['no credits taken/ all credits paid back duly', 'all credits at this bank paid back duly', 'existing credits paid back duly till now']:
            return 'None/Paid'
        elif x == 'delay in paying off in the past':
            return 'Delay'
        elif x == 'critical account/ other credits existing (not at this bank)':
            return 'Other'
        else:
            return 'NA'

    def group_employ(x):
        if x == 'unemployed':
            return 'Unemployed'
        elif x in ['... < 1 year ', '1 <= ... < 4 years']:
            return '1-4 years'
        elif x in ['4 <= ... < 7 years', '.. >= 7 years']:
            return '4+ years'
        else:
            return 'NA'

    def group_savings(x):
        if x in ['... < 100 DM', '100 <= ... < 500 DM']:
            return '<500'
        elif x in ['500 <= ... < 1000 DM ', '.. >= 1000 DM ']:
            return '500+'
        elif x == 'unknown/ no savings account':
            return 'Unknown/None'
        else:
            return 'NA'

    def group_status(x):
        if x in ['< 0 DM', '0 <= ... < 200 DM']:
            return '<200'
        elif x in ['>= 200 DM / salary assignments for at least 1 year']:
            return '200+'
        elif x == 'no checking account':
            return 'None'
        else:
            return 'NA'

    status_map = {'male : divorced/separated': 1.0,
                'male : single': 1.0,
                'male : married/widowed': 1.0,
                'female : divorced/separated/married': 0.0,
                'female : single': 0.0}

    df['personal_status_sex'] = df['personal_status_sex'].replace(status_map)
    df['credit_history'] = df['credit_history'].apply(lambda x: group_credit_hist(x))
    df['savings'] = df['savings'].apply(lambda x: group_savings(x))
    df['present_emp_since'] = df['present_emp_since'].apply(lambda x: group_employ(x))
    df['age'] = df['age'].apply(lambda x: np.float(x >= 25))
    df['account_check_status'] = df['account_check_status'].apply(lambda x: group_status(x))

    df = df.rename(columns = {'default': 'credit', 'present_emp_since': 'employment', 'account_check_status': 'status', 'personal_status_sex': 'sex'})

    protected_attribute = ['sex', 'age']
    label_name = 'credit'
    categorical_features = ['credit_history', 'savings', 'employment']
    features = categorical_features + [label_name] + protected_attribute

    privileged_class = {'sex': [1.0], 'age': [1.0]}

    protected_attribute_map = {"sex": {1.0: 'male', 0.0: 'female'},
                            "age": {1.0: 'old', 0.0: 'young'}}

    data = StandardDataset(df, label_name, favorable_classes=[1],
                            protected_attribute_names=protected_attribute,
                            privileged_classes=[privileged_class[x] for x in protected_attribute],
                            categorical_features=categorical_features,
                            features_to_keep=features,
                            metadata={'label_maps': [{1.0: 'Good Credit', 2.0: 'Bad Credit'}],
                                    'protected_attribute_maps': [protected_attribute_map[x] for x in protected_attribute]})

    return data

In [30]:
# for i in range(0, num_runs):
#     print("starting", i)
    ###########################################
    
    privileged_groups = [{'sex': 1}]
    unprivileged_groups = [{'sex': 0}]
    # change this line to run it on different datasets
#     dataset_orig = load_preproc_data_german()
    df = pd.read_csv('dataset/german_credit.csv')
    dataset_orig = preprocess_germandataset(df)
    # train1, test1 are the original dataset
    train1, test1 = dataset_orig.split([0.7], shuffle=True)
    
    print("splitting data complete")
    
    ###########################################

    RW = Reweighing(unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
    RW.fit(train1)
    # dataset_transf_train, test1 are for the reweighed dataset
    dataset_transf_train = RW.transform(train1)

    # change weights to whole numbers
    for i in range(dataset_transf_train.instance_weights.size):
        dataset_transf_train.instance_weights[i] = (round(dataset_transf_train.instance_weights[i] / 0.1) * 0.1) * 10
        weights = copy.deepcopy(dataset_transf_train.instance_weights)

    # change dataset_transf_train.features and dataset_transf_train.labels and dataset_transf_train.protected_attributes according to the weights of each instance
    sum_weights = 0
    for i in range(dataset_transf_train.features.shape[0]):
        row = copy.deepcopy(dataset_transf_train.features[i])
        row_label = copy.deepcopy(dataset_transf_train.labels[i])
        row_protected_attributes = copy.deepcopy(dataset_transf_train.protected_attributes[i])
        row_protected_attributes.resize(1,2)
        row.resize(1,dataset_transf_train.features.shape[1])
        row_label.resize(1,1)
        weight = int(weights[i])
        for j in range(weight-1):
            dataset_transf_train.features = np.concatenate((dataset_transf_train.features,row))
            dataset_transf_train.labels = np.concatenate((dataset_transf_train.labels,row_label))
            dataset_transf_train.protected_attributes = np.concatenate((dataset_transf_train.protected_attributes,row_protected_attributes))

    # change the dataset_transf_train to a numpy array of ones to match number of rows in features
    dataset_transf_train.instance_weights = np.ones(dataset_transf_train.features.shape[0])

    print("reweighing complete\n")

    ################## with reweighing ##########################

    temp_stat_par = []
    temp_disp_imp = []
    temp_eq_opp_diff = []
    temp_avg_odds_diff = []
    temp_theil = []
    temp_acc = []

    ################## adversarial debiasing #################

    sess = tf.Session()
    debiased_model_reweighing = AdversarialDebiasing(privileged_groups = privileged_groups,
        unprivileged_groups = unprivileged_groups,
        scope_name='debiased_classifier',
        debias=True,
        sess=sess)
    debiased_model_reweighing.fit(dataset_transf_train)
    dataset_debiasing_test_reweighing = debiased_model_reweighing.predict(test1)
    sess.close()
    tf.reset_default_graph()

    print("  Adversarial Debiasing")

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_debiasing_test_reweighing, 
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test1,
        dataset_debiasing_test_reweighing,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
#     temp_stat_par.append(metric_test.mean_difference())
#     temp_disp_imp.append(acc_test.disparate_impact())
#     temp_eq_opp_diff.append(acc_test.equal_opportunity_difference())
#     temp_avg_odds_diff.append(acc_test.average_odds_difference())
#     temp_theil.append(acc_test.theil_index())
#     temp_acc.append(accuracy_score(y_true=test1.labels, y_pred=dataset_debiasing_test_reweighing.labels))
    
    
    print('Mean difference = {}'.format(metric_test.mean_difference()))
    print('Disparate impact = {}'.format(acc_test.disparate_impact()))
    print('Equal Opportunity Difference = {}'.format(acc_test.equal_opportunity_difference()))
    print('Average Odds Difference = {}'.format(acc_test.average_odds_difference()))
    print('Theil index = {}'.format(acc_test.theil_index()))
    print('Accuracy = {:.2f}%'.format(accuracy_score(y_true=test1.labels, y_pred=dataset_debiasing_test_reweighing.labels) * 100))
    print('\n')
#     print("metrics for adv debiasing complete")

    ##################### prejudice remover #####################
    prejudice_model_reweighing = PrejudiceRemover(eta=100, sensitive_attr='sex')
    prejudice_model_reweighing.fit(dataset_transf_train)
    dataset_prejudice_test_reweighing = prejudice_model_reweighing.predict(test1)

    print("  Prejudice Remover")

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_prejudice_test_reweighing, 
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test1,
        dataset_prejudice_test_reweighing,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
#     temp_stat_par.append(metric_test.mean_difference())
#     temp_disp_imp.append(acc_test.disparate_impact())
#     temp_eq_opp_diff.append(acc_test.equal_opportunity_difference())
#     temp_avg_odds_diff.append(acc_test.average_odds_difference())
#     temp_theil.append(acc_test.theil_index())
#     temp_acc.append(accuracy_score(y_true=test1.labels, y_pred=dataset_prejudice_test_reweighing.labels))
    
    print('Mean difference = {}'.format(metric_test.mean_difference()))
    print('Disparate impact = {}'.format(acc_test.disparate_impact()))
#     print('Equal Opportunity Difference = {}'.format(acc_test.equal_opportunity_difference()))
#     print('Average Odds Difference = {}'.format(acc_test.average_odds_difference()))
    print('Theil index = {}'.format(acc_test.theil_index()))
    print('Accuracy = {:.2f}%'.format(accuracy_score(y_true=test1.labels, y_pred=dataset_prejudice_test_reweighing.labels) * 100))
    print("\n")

    ##################### normal neural net #####################
    sess = tf.Session()
    neural_model = AdversarialDebiasing(privileged_groups = privileged_groups,
        unprivileged_groups = unprivileged_groups,
        scope_name='debiased_classifier',
        debias=False,
        sess=sess)
    neural_model.fit(dataset_transf_train)
    dataset_neural_test = neural_model.predict(test1)
    sess.close()
    tf.reset_default_graph()

    print("  Non-debiasing")

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_neural_test, 
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test1,
        dataset_neural_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
#     temp_stat_par.append(metric_test.mean_difference())
#     temp_disp_imp.append(acc_test.disparate_impact())
#     temp_eq_opp_diff.append(acc_test.equal_opportunity_difference())
#     temp_avg_odds_diff.append(acc_test.average_odds_difference())
#     temp_theil.append(acc_test.theil_index())
#     temp_acc.append(accuracy_score(y_true=test1.labels, y_pred=dataset_neural_test.labels))
    
    print('Mean difference = {}'.format(metric_test.mean_difference()))
    print('Disparate impact = {}'.format(acc_test.disparate_impact()))
    print('Equal Opportunity Difference = {}'.format(acc_test.equal_opportunity_difference()))
    print('Average Odds Difference = {}'.format(acc_test.average_odds_difference()))
    print('Theil index = {}'.format(acc_test.theil_index()))
    print('Accuracy = {:.2f}%'.format(accuracy_score(y_true=test1.labels, y_pred=dataset_neural_test.labels) * 100))

    print("\n")


    ##################### ensemble #####################
    pred_labels_test = []
    for i in range(0, len(test1.features)):
        arr_test = mode([dataset_debiasing_test_reweighing.labels[i], dataset_prejudice_test_reweighing.labels[i], dataset_neural_test.labels[i]])
        pred_labels_test.append(arr_test[0][0])
        dataset_ensemble_test = test1.copy()
        dataset_ensemble_test.labels = np.array(pred_labels_test)

    print("  Ensemble")

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_ensemble_test, 
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test1,
        dataset_ensemble_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
#     temp_stat_par.append(metric_test.mean_difference())
#     temp_disp_imp.append(acc_test.disparate_impact())
#     temp_eq_opp_diff.append(acc_test.equal_opportunity_difference())
#     temp_avg_odds_diff.append(acc_test.average_odds_difference())
#     temp_theil.append(acc_test.theil_index())
#     temp_acc.append(accuracy_score(y_true=test1.labels, y_pred=dataset_ensemble_test.labels))
    
    print('Mean difference = {}'.format(metric_test.mean_difference()))
    print('Disparate impact = {}'.format(acc_test.disparate_impact()))
    print('Equal Opportunity Difference = {}'.format(acc_test.equal_opportunity_difference()))
    print('Average Odds Difference = {}'.format(acc_test.average_odds_difference()))
    print('Theil index = {}'.format(acc_test.theil_index()))
    print('Accuracy = {:.2f}%'.format(accuracy_score(y_true=test1.labels, y_pred=dataset_ensemble_test.labels) * 100))

    print("\n")


#     ######### DUMP SHIT ###########

#     stat_par_reweigh.append(temp_stat_par)
#     disp_imp_reweigh.append(temp_disp_imp)
#     eq_opp_diff_reweigh.append(temp_eq_opp_diff)
#     avg_odds_diff_reweigh.append(temp_avg_odds_diff)
#     theil_reweigh.append(temp_theil)
#     acc_reweigh.append(temp_acc)

#     print("compiling stuff for reweighing complete")

    print("compiling all metrics complete")
    
#     print("ending", i)
#     print("\n")

splitting data complete
reweighing complete

  Adversarial Debiasing
Mean difference = 0.1
Disparate impact = 2.666666666666667
Equal Opportunity Difference = 0.19242424242424244
Average Odds Difference = 0.12377075660657752
Theil index = 0.34448208450794915
Accuracy = 67.00%


  Prejudice Remover
Mean difference = -0.04
Disparate impact = 0.0
Theil index = 0.37827961177430214
Accuracy = 67.00%


  Non-debiasing
Mean difference = 0.07
Disparate impact = 2.166666666666667
Equal Opportunity Difference = 0.19242424242424244
Average Odds Difference = 0.10138269690508497
Theil index = 0.3407007971954018
Accuracy = 68.00%


  Ensemble
Mean difference = 0.07
Disparate impact = 2.166666666666667
Equal Opportunity Difference = 0.19242424242424244
Average Odds Difference = 0.10138269690508497
Theil index = 0.3407007971954018
Accuracy = 68.00%


compiling all metrics complete


In [35]:
# for i in range(0, num_runs):
#     print("starting", i)
    ###########################################
    
    privileged_groups = [{'sex': 1}]
    unprivileged_groups = [{'sex': 0}]
    # change this line to run it on different datasets
#     dataset_orig = load_preproc_data_german()
    df = pd.read_csv('dataset/german_credit.csv')
    dataset_orig = preprocess_germandataset(df)
    # train1, test1 are the original dataset
    train1, test1 = dataset_orig.split([0.7], shuffle=True)
    
    print("splitting data complete\n")    
    ################## without reweighing ##########################

    temp_stat_par = []
    temp_disp_imp = []
    temp_eq_opp_diff = []
    temp_avg_odds_diff = []
    temp_theil = []
    temp_acc = []

    ##################### adversarial debiasing #####################

    sess = tf.Session()
    debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
        unprivileged_groups = unprivileged_groups,
        scope_name='debiased_classifier',
        debias=True,
        sess=sess)
    debiased_model.fit(train1)
    dataset_debiasing_test = debiased_model.predict(test1)
    sess.close()
    tf.reset_default_graph()
    print("  Adversarial Debiasing")

    ##################### metrics #####################

    acc_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test1,
        dataset_debiasing_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
#     temp_stat_par.append(metric_test.mean_difference())
#     temp_disp_imp.append(acc_test.disparate_impact())
#     temp_eq_opp_diff.append(acc_test.equal_opportunity_difference())
#     temp_avg_odds_diff.append(acc_test.average_odds_difference())
#     temp_theil.append(acc_test.theil_index())
#     temp_acc.append(accuracy_score(y_true=test1.labels, y_pred=dataset_debiasing_test.labels))
    
    print('Mean difference = {}'.format(metric_test.mean_difference()))
    print('Disparate impact = {}'.format(acc_test.disparate_impact()))
    print('Equal Opportunity Difference = {}'.format(acc_test.equal_opportunity_difference()))
    print('Average Odds Difference = {}'.format(acc_test.average_odds_difference()))
    print('Theil index = {}'.format(acc_test.theil_index()))
    print('Accuracy = {:.2f}%'.format(accuracy_score(y_true=test1.labels, y_pred=dataset_debiasing_test.labels) * 100))

    print("\n")

    ##################### prejudice remover #####################

    prejudice_model = PrejudiceRemover(eta=100, sensitive_attr='sex')
    prejudice_model.fit(train1)
    dataset_prejudice_test = prejudice_model.predict(test1)

    print("  Prejudice Remover")

    ##################### metrics #####################

    acc_test = BinaryLabelDatasetMetric(dataset_prejudice_test, 
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test1,
        dataset_prejudice_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
#     temp_stat_par.append(metric_test.mean_difference())
#     temp_disp_imp.append(acc_test.disparate_impact())
#     temp_eq_opp_diff.append(acc_test.equal_opportunity_difference())
#     temp_avg_odds_diff.append(acc_test.average_odds_difference())
#     temp_theil.append(acc_test.theil_index())
#     temp_acc.append(accuracy_score(y_true=test1.labels, y_pred=dataset_prejudice_test.labels))
    
    print('Mean difference = {}'.format(metric_test.mean_difference()))
    print('Disparate impact = {}'.format(acc_test.disparate_impact()))
#     print('Equal Opportunity Difference = {}'.format(acc_test.equal_opportunity_difference()))
#     print('Average Odds Difference = {}'.format(acc_test.average_odds_difference()))
    print('Theil index = {}'.format(acc_test.theil_index()))
    print('Accuracy = {:.2f}%'.format(accuracy_score(y_true=test1.labels, y_pred=dataset_prejudice_test.labels) * 100))

    print("\n")

    ##################### normal neural net #####################

    sess = tf.Session()
    neural_model = AdversarialDebiasing(privileged_groups = privileged_groups,
        unprivileged_groups = unprivileged_groups,
        scope_name='debiased_classifier',
        debias=False,
        sess=sess)
    neural_model.fit(train1)
    dataset_neural_test = neural_model.predict(test1)
    sess.close()
    tf.reset_default_graph()

    print("  Non-debiasing")

    ##################### metrics #####################

    acc_test = BinaryLabelDatasetMetric(dataset_neural_test, 
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test1,
        dataset_neural_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
#     temp_stat_par.append(metric_test.mean_difference())
#     temp_disp_imp.append(acc_test.disparate_impact())
#     temp_eq_opp_diff.append(acc_test.equal_opportunity_difference())
#     temp_avg_odds_diff.append(acc_test.average_odds_difference())
#     temp_theil.append(acc_test.theil_index())
#     temp_acc.append(accuracy_score(y_true=test1.labels, y_pred=dataset_neural_test.labels))
    
    print('Mean difference = {}'.format(metric_test.mean_difference()))
    print('Disparate impact = {}'.format(acc_test.disparate_impact()))
    print('Equal Opportunity Difference = {}'.format(acc_test.equal_opportunity_difference()))
    print('Average Odds Difference = {}'.format(acc_test.average_odds_difference()))
    print('Theil index = {}'.format(acc_test.theil_index()))
    print('Accuracy = {:.2f}%'.format(accuracy_score(y_true=test1.labels, y_pred=dataset_neural_test.labels) * 100))

    print("\n")

    ##################### ensemble #####################

    pred_labels_test = []
    for i in range(0, len(test1.features)):
        arr_test = mode([dataset_debiasing_test.labels[i], dataset_prejudice_test.labels[i], dataset_neural_test.labels[i]])
        pred_labels_test.append(arr_test[0][0])
        dataset_ensemble_test = test1.copy()
        dataset_ensemble_test.labels = np.array(pred_labels_test)

    print("  Ensemble")

    ##################### metrics #####################

    acc_test = BinaryLabelDatasetMetric(dataset_ensemble_test, 
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test1,
        dataset_ensemble_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
#     temp_stat_par.append(metric_test.mean_difference())
#     temp_disp_imp.append(acc_test.disparate_impact())
#     temp_eq_opp_diff.append(acc_test.equal_opportunity_difference())
#     temp_avg_odds_diff.append(acc_test.average_odds_difference())
#     temp_theil.append(acc_test.theil_index())
#     temp_acc.append(accuracy_score(y_true=test1.labels, y_pred=dataset_ensemble_test.labels))
    
    print('Mean difference = {}'.format(metric_test.mean_difference()))
    print('Disparate impact = {}'.format(acc_test.disparate_impact()))
    print('Equal Opportunity Difference = {}'.format(acc_test.equal_opportunity_difference()))
    print('Average Odds Difference = {}'.format(acc_test.average_odds_difference()))
    print('Theil index = {}'.format(acc_test.theil_index()))
    print('Accuracy = {:.2f}%'.format(accuracy_score(y_true=test1.labels, y_pred=dataset_ensemble_test.labels) * 100))

    print("\n")

    ######### DUMP SHIT ###########

#     stat_par.append(temp_stat_par)
#     disp_imp.append(temp_disp_imp)
#     eq_opp_diff.append(temp_eq_opp_diff)
#     avg_odds_diff.append(temp_avg_odds_diff)
#     theil.append(temp_theil)
#     acc.append(temp_acc)

    print("compiling all metrics complete")

splitting data complete

  Adversarial Debiasing
Mean difference = 0.07
Disparate impact = 1.9586206896551726
Equal Opportunity Difference = 0.12301587301587302
Average Odds Difference = 0.05220357782178689
Theil index = 0.3261857963035578
Accuracy = 70.67%


  Prejudice Remover
Mean difference = 0.07
Disparate impact = 0.0
Theil index = 0.3716346875467681
Accuracy = 68.33%


  Non-debiasing
Mean difference = 0.07
Disparate impact = 6.528735632183908
Equal Opportunity Difference = 0.19444444444444445
Average Odds Difference = 0.09747200366346113
Theil index = 0.3400311897916616
Accuracy = 70.33%


  Ensemble
Mean difference = 0.07
Disparate impact = 6.528735632183908
Equal Opportunity Difference = 0.19444444444444445
Average Odds Difference = 0.09747200366346113
Theil index = 0.3400311897916616
Accuracy = 70.33%


compiling all metrics complete


In [11]:
########## WITHOUT REWEIGHING #############
print(stat_par)
print(disp_imp)
print(eq_opp_diff)
print(avg_odds_diff)
print(theil)
print(acc)

########## WITH REWEIGHING #############
print("\n")
print(stat_par_reweigh)
print(disp_imp_reweigh)
print(eq_opp_diff_reweigh)
print(avg_odds_diff_reweigh)
print(theil_reweigh)
print(acc_reweigh)

[[-0.03825417201540437, -0.03825417201540437, -0.03825417201540437, -0.03825417201540437], [0.07983342643847442, 0.07983342643847442, 0.07983342643847442, 0.07983342643847442], [0.059097127222982215, 0.059097127222982215, 0.059097127222982215, 0.059097127222982215], [0.15257352941176472, 0.15257352941176472, 0.15257352941176472, 0.15257352941176472]]
[[4.046052631578947, 0.0, 30.210526315789473, 15.105263157894736], [inf, nan, inf, inf], [0.0, 0.0, nan, nan], [0.0, nan, 12.041666666666668, 0.0]]
[[], [], [], []]
[[], [], [], []]
[[0.3305973427665854, 0.36681357301890394, 0.3432567241190788, 0.3448170771435578], [0.2525843512124688, 0.31015492830383956, 0.291371135587219, 0.291371135587219], [0.37503243009982523, 0.3807724955177929, 0.38566248081198456, 0.38566248081198456], [0.17887637150250923, 0.3955147772549962, 0.36355078472625907, 0.39429731855310884]]
[[0.6966666666666667, 0.6866666666666666, 0.6966666666666667, 0.6933333333333334], [0.7433333333333333, 0.7333333333333333, 0.7366

In [ ]:
without_reweighing = [stat_par,disp_imp,theil,acc]
with_reweighing = [stat_par_reweigh,disp_imp_reweigh,theil_reweigh,acc_reweigh]

for metric in range(len(without_reweighing)):
    name = "metric" + str(metric) + ".csv"
    sublist = without_reweighing[metric]
    with open(name, "w") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(sublist)

for metric in range(len(with_reweighing)):
    name = "metric" + str(metric) + "reweigh.csv"
    sublist = with_reweighing[metric]
    with open(name, "w") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(sublist)